In [1]:
!pip install datasets scikit-learn pandas transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.4 MB/s eta 0:00:00


In [3]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [4]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np
import evaluate

In [5]:
# === Step 1: Load Your Data ===
train_df_1 = pd.read_excel("/content/incidents_train_clean_excel.xlsx")
test_df_1 = pd.read_excel("/content/incidents_test_clean_excel.xlsx")

train_df=train_df_1[["title", "product-category"]]
test_df=test_df_1[["title", "product-category"]]

In [6]:
# === Step 2: Encode Labels (if not already integers) ===
label_encoder = LabelEncoder()

#test_df['label'] = label_encoder.transform(test_df['hazard'])

train_df['label'] = label_encoder.fit_transform(train_df['product-category'])

# Filter test_df to avoid unseen labels
test_df = test_df[test_df['product-category'].isin(label_encoder.classes_)]
test_df['label'] = label_encoder.transform(test_df['product-category'])

num_labels = len(label_encoder.classes_)
num_labels

<ipython-input-6-085a3b8e340b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['label'] = label_encoder.fit_transform(train_df['product-category'])
<ipython-input-6-085a3b8e340b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['label'] = label_encoder.transform(test_df['product-category'])


24

In [7]:
print("Label range:", train_df['label'].min(), "to", train_df['label'].max())
print("Number of classes (num_labels):", num_labels)


Label range: 0 to 23
Number of classes (num_labels): 24


In [8]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [9]:
# === Step 4: Tokenize ===
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(example):
    return tokenizer(example['title'], truncation=True, padding='max_length')

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/5084 [00:00<?, ? examples/s]

Map:   0%|          | 0/997 [00:00<?, ? examples/s]

In [10]:
train_dataset.set_format("torch")
test_dataset.set_format("torch")

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0
    )

    macro_f1 = precision_recall_fscore_support(
        labels, preds, average='macro', zero_division=0
    )[2]

    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'weighted_precision': precision,
        'weighted_recall': recall,
        'weighted_f1': f1,
        'macro_f1': macro_f1,
    }


In [13]:
training_args = TrainingArguments(
    output_dir="./distilbert-results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=4,
    weight_decay=0.01,
    report_to="none",  # Disable wandb and other logging tools
    logging_dir="./logs",  # Directory to store TensorBoard logs
    logging_steps=10,  # Log every 10 steps
    eval_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",  # Save the model at the end of each epoch
    load_best_model_at_end=True,  # Load the best model based on evaluation metric
    metric_for_best_model="weighted_f1",  # Choose F1-score as the metric for best model
    logging_first_step=True,  # Log the first step as well
)

In [14]:
# === Step 8: Initialize Trainer ===
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-14-b92ff7ee9bf7>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Weighted Precision,Weighted Recall,Weighted F1,Macro F1
1,1.538000,1.348795,0.644935,0.626578,0.644935,0.592403,0.313195
2,1.106100,1.059985,0.728185,0.710890,0.728185,0.705098,0.435837
3,0.835100,0.989736,0.743230,0.721499,0.743230,0.726919,0.463569
4,0.683900,0.967772,0.752257,0.729152,0.752257,0.737212,0.476485


TrainOutput(global_step=1272, training_loss=1.1867857991524462, metrics={'train_runtime': 1009.5167, 'train_samples_per_second': 20.144, 'train_steps_per_second': 1.26, 'total_flos': 2694913923612672.0, 'train_loss': 1.1867857991524462, 'epoch': 4.0})

In [ ]:


!zip -r my_model.zip /content/distilbert-results/checkpoint-954


  adding: content/distilbert-results/checkpoint-954/ (stored 0%)
  adding: content/distilbert-results/checkpoint-954/rng_state.pth (deflated 25%)
  adding: content/distilbert-results/checkpoint-954/special_tokens_map.json (deflated 42%)
  adding: content/distilbert-results/checkpoint-954/training_args.bin (deflated 51%)
  adding: content/distilbert-results/checkpoint-954/tokenizer.json (deflated 71%)
  adding: content/distilbert-results/checkpoint-954/optimizer.pt (deflated 34%)
  adding: content/distilbert-results/checkpoint-954/scheduler.pt (deflated 56%)
  adding: content/distilbert-results/checkpoint-954/vocab.txt (deflated 53%)
  adding: content/distilbert-results/checkpoint-954/tokenizer_config.json (deflated 75%)
  adding: content/distilbert-results/checkpoint-954/trainer_state.json (deflated 79%)
  adding: content/distilbert-results/checkpoint-954/model.safetensors (deflated 8%)
  adding: content/distilbert-results/checkpoint-954/config.json (deflated 75%)


In [ ]:
from google.colab import files
files.download("my_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>